In [4]:
import pandas as pd
import numpy as np

In [8]:
neutral_test = pd.DataFrame({'neutral' : [1,0,1.000001,0.999999,1.0],
                                 'compound' : [1, 1, 1, 1, 1]})

neutral_pass = pd.DataFrame({'neutral' : [np.nan, 0, 1.000001, 0.999999, np.nan],
                                 'compound': [np.nan, 1, 1 ,1, np.nan]})

In [9]:
neutral_pass

,compound,neutral
0,NaN,NaN
1,1.0,0.000000
2,1.0,1.000001
3,1.0,0.999999
4,NaN,NaN


In [10]:
neutral_test

,compound,neutral
0,1,1.000000
1,1,0.000000
2,1,1.000001
3,1,0.999999
4,1,1.000000


In [29]:
def test_resample_dataframe():

    df = pd.DataFrame()
    output_df = pd.DataFrame()

    df['sent_at'] = ['2016-05-01 12:00:00','2016-05-01 12:01:00',
                     '2016-05-01 12:02:00','2016-05-01 12:03:00',
                     '2016-06-02','2016-06-03','2016-07-02','2016-07-03',
                     '2016-09-01 12:00:00','2016-09-01 12:00:01',
                     '2016-09-01 12:00:02','2016-09-01 12:00:03',
                     '2016-10-01','2016-10-02']

    df['response_time'] = [0, 60, 60, 60, 0, 0, 0, 0, 0, 60, 60, 60, 0, 0]

    df['to_from'] = [True, False, True, False, True, True, True, True, True,
                     False, True, False, True, True]

    df['positive'] = [0,0,0,0,0,0,0,0,0.5,0.2,0.2,0.3,0.1,0.1]

    df['negative'] = [0,0,0,0,0,0,0,0,0.5,0.9,0.9,0.3,0.1,0.2]

    df['neutral'] = [ 0,0,0,0,0,0,0,0,0.5,0.5,0.2,0.3,0.1,0.1]

    df['compound'] = [0,0,0,0,0,0,0,0,0.5,-0.5,-0.3,0.3,0.1,-0.1]

    df['word_count'] = [20,0,10,2,10,10,0,0,0,0,0,0,0,0]



    output_df['message_count_user'] = [0.5,0.25,0.25,0,0,0]

    output_df['message_count_contact'] = [1,0,0,0,0,0]

    output_df['message_count'] = [0.6,0.2,0.2,0,0,0]

    output_df['word_count'] = [52/72,20/72,0,0,0,0]

    output_df['word_count_user'] = [5/7, 2/7, 0, 0,0,0]

    output_df['word_count_contact'] = [1,0,0,0,0,0]

    output_df['message_count_reciprocity'] = [0.5,0,0,0,0,0]

    output_df['word_count_reciprocity'] = [0.04,0,0,0,0,0]

    output_df['positive'] = [0,0,0,0,0.3,0.1]

    output_df['negative'] = [0,0,0,0,0.65,0.15]

    output_df['neutral'] = [0,0,0,0,0.375,0.1]

    output_df['compound'] = [0,0,0,0,0,0]

    output_df['positive_user'] = [0,0,0,0,0.35,0.1]

    output_df['negative_user'] = [0,0,0,0,0.7,0.15]

    output_df['neutral_user'] = [0,0,0,0,0.35,0.1]

    output_df['compound_user'] = [0,0,0,0,0.1,0]

    output_df['positive_contact'] = [0,0,0,0,0.25,0]

    output_df['negative_contact'] = [0,0,0,0,0.6,0]

    output_df['neutral_contact'] = [0,0,0,0,0.4,0]

    output_df['compound_contact'] = [0,0,0,0,-0.1,0]

    output_df['sentiment_reciprocity'] = [0,0,0,0,0.2,0]

    output_df["response_time"] = [60, 0, 0, 0, 60, 0]
    output_df["response_time_user"] = [60, 0, 0, 0, 60, 0]
    output_df["response_time_contacts"] = [60, 0, 0, 0, 60, 0]

    assert resample_dataframe(df, 'M').equals(output_df)


In [35]:
test_resample_dataframe()

NameError: global name 'config' is not defined

In [31]:
def resample_dataframe(df, period='D'):
    '''
    This function resamples the features
    in the dataframe to the defined timeseries

    It create reciprocity features based on this
    resampling

    :param df: the dataframe to be resampled
    :param period: the resampling time interval
    :return: df with the resampled features
    '''

    #TODO perhaps this cleaning of sentiment should be outside the resampling of the dataframe
    df = sentiment_cleaning(df)

    time_field = 'sent_at'
    if isinstance(df[time_field].tolist()[0], str):
        df[time_field] = pd.to_datetime(df[time_field])
    else:
        print("nothing to be done")

    output_df = pd.DataFrame()

    # get the message counts total and for the user and contact
    output_df["message_count"] = df[time_field].value_counts().resample(period).apply(np.nansum)
    output_df["message_count_not_normalised"] = output_df["message_count"].copy()
    output_df["message_count_user"] = df[~df["to_from"]][time_field].value_counts().resample(period).apply(np.nansum)
    output_df["message_count_contact"] = df[df["to_from"]][time_field].value_counts().resample(period).apply(np.nansum)

    df.set_index(time_field, inplace=True, drop=False)

    # get the total word counts and for the user and contact
    keys = ["word_count"]
    for k in keys:
        output_df[k] = df[k].resample(period).apply(np.nansum)
        output_df[k + "_user"] = df[~df["to_from"]][k].resample(period).apply(np.nansum)
        output_df[k + "_contact"] = df[df["to_from"]][k].resample(period).apply(np.nansum)

    # get the sentiments overall and for the user and contact individually
    keys = ["positive", "negative", "neutral", "compound"]
    for k in keys:
        output_df[k] = df[k].resample(period).apply(np.nanmean)
        output_df[k + "_user"] = df[~df["to_from"]][k].resample(period).apply(np.nanmean)
        output_df[k + "_contact"] = df[df["to_from"]][k].resample(period).apply(np.nanmean)

    # get the time differences
    output_df["response_time"] = df["response_time"].resample(period).apply(_average_time)
    output_df["response_time_user"] = df[~df["to_from"]]["response_time"].resample(period).apply(_average_time)
    output_df["response_time_contact"] = df[df["to_from"]]["response_time"].resample(period).apply(_average_time)

    # now compute reciprocity between users
    output_df["sentiment_reciprocity"] = (output_df["compound_contact"] - output_df["compound_user"]).abs()
    for k in ["message_count", "word_count", "response_time"]:
        output_df[k + "_reciprocity"] = find_ratio(output_df, k)

    # now normalise number of mesages, words, etc
    keys = ["word_count", "message_count",
            "message_count_user","message_count_contact",
            "word_count_user","word_count_contact"]
    for k in keys:
        output_df[k] = output_df[k] / (output_df[k].sum(axis=0))

    # compute the relative difference of the response time over the course of the relationship
    keys = ["response_time", "response_time_user", "response_time_contact"]
    for k in keys:
        if output_df[k].dropna().size == 0:
            denom = 1
        else:
            denom = output_df[k].dropna().size
        mean_time = output_df[k].sum(axis=0) / denom  # don't want to average with null months
        output_df[k] = (output_df[k] - mean_time) / mean_time  # evals to 0 if avg. time is == time_limit
    return output_df

In [34]:
def sentiment_cleaning(df):
    """Where neutral = 1 or if +ve, -ve
    and neutral = 0 then replace with nan"""

    mask_neutral = df["neutral"] == 1.0
    mask_zeroes = df[['positive', 'negative', 'neutral']].sum(axis=1) == 0

    for col in ['positive', 'negative', 'neutral', 'compound']:
        df[col].where(~mask_neutral, np.nan, inplace=True)
        df[col].where(~mask_zeroes, np.nan, inplace=True)
    return df

def _average_time(x):

    #TODO we may need to tweak the time limit
    if config.model == 'not_romantic':
        time_limit = config.resampler['response_time_limit_not_romantic'] * 3600  # in seconds
    elif config.model == 'romantic':
        time_limit = config.resampler['response_time_limit_romantic'] * 3600  # in seconds

    x_mean = np.nanmean(x)  # if all nans returns nan
    if x_mean > time_limit:
        return time_limit
    else:
        return x_mean
